In [ ]:
import os

# create folder structure inside Colab runtime
os.makedirs("/content/data", exist_ok=True)
os.makedirs("/content/outputs", exist_ok=True)
os.makedirs("/content/scripts", exist_ok=True)

print("✅  Folders created successfully!")


✅  Folders created successfully!


In [ ]:
import pandas as pd
import sqlite3
import os

url = "https://raw.githubusercontent.com/salemprakash/EDA/main/Data/constituents.csv"
df = pd.read_csv(url)
print("✅ Dataset loaded\n")
print(df.head())

conn = sqlite3.connect("/content/data/constituents.db")
df.to_sql("constituents", conn, if_exists="replace", index=False)
conn.commit()
print("✅ Database created at /content/data/constituents.db")

query1 = "SELECT COUNT(*) AS total_companies FROM constituents;"
query2 = "SELECT [GICS Sector], COUNT(*) AS count FROM constituents GROUP BY [GICS Sector];"
query3 = "SELECT [GICS Sub-Industry], COUNT(*) AS count FROM constituents GROUP BY [GICS Sub-Industry] ORDER BY count DESC LIMIT 10;"

total_companies = pd.read_sql(query1, conn)
sector_counts = pd.read_sql(query2, conn)
subindustry_counts = pd.read_sql(query3, conn)

total_companies.to_csv("/content/outputs/sql_total_companies.csv", index=False)
sector_counts.to_csv("/content/outputs/sql_sector_counts.csv", index=False)
subindustry_counts.to_csv("/content/outputs/sql_top10_subindustries.csv", index=False)

sql_text = """-- Main SQL Queries
SELECT COUNT(*) AS total_companies FROM constituents;
SELECT [GICS Sector], COUNT(*) AS count FROM constituents GROUP BY [GICS Sector];
SELECT [GICS Sub-Industry], COUNT(*) AS count FROM constituents GROUP BY [GICS Sub-Industry] ORDER BY count DESC LIMIT 10;
"""
with open("/content/scripts/main_sql_queries.sql", "w") as f:
    f.write(sql_text)

print("✅ All query outputs and SQL file saved successfully.")



✅ Dataset loaded

  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  
✅ Database created at /content/data/co

In [ ]:
import datetime
from sklearn.preprocessing import StandardScaler

df_clean = df.copy()

df_clean['Founded_Year'] = pd.to_numeric(df_clean['Founded'], errors='coerce')
df_clean['Age'] = datetime.datetime.now().year - df_clean['Founded_Year']

df_clean['Date added'] = pd.to_datetime(df_clean['Date added'], errors='coerce')
df_clean['Years_Since_Added'] = datetime.datetime.now().year - df_clean['Date added'].dt.year

df_clean['HQ_Country'] = df_clean['Headquarters Location'].astype(str).str.split(',').str[-1].str.strip()

df_clean = df_clean.dropna(subset=['Founded_Year', 'Years_Since_Added', 'GICS Sector', 'GICS Sub-Industry', 'HQ_Country'])

df_encoded = pd.get_dummies(df_clean[['GICS Sector','GICS Sub-Industry','HQ_Country']], drop_first=True)

num_features = df_clean[['Age','Years_Since_Added']]
X = pd.concat([num_features, df_encoded], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Feature engineering & scaling complete")
print("Shape of final matrix:", X_scaled.shape)


✅ Feature engineering & scaling complete
Shape of final matrix: (461, 178)


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/salemprakash/EDA/main/Data/constituents.csv"
df_check = pd.read_csv(url)

print("✅ Dataset loaded successfully!\n")
print("Columns:\n", df_check.columns.tolist(), "\n")

print("Shape:", df_check.shape, "\n")
print("Missing values per column:\n", df_check.isna().sum(), "\n")

print("Sample rows:\n")
display(df_check.head(10))


✅ Dataset loaded successfully!

Columns:
 ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded'] 

Shape: (503, 8) 

Missing values per column:
 Symbol                   0
Security                 0
GICS Sector              0
GICS Sub-Industry        0
Headquarters Location    0
Date added               0
CIK                      0
Founded                  0
dtype: int64 

Sample rows:



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
6,AMD,Advanced Micro Devices,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
7,AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981
8,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
9,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",2000-06-05,1090872,1999


In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=10)
pca_result = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(10)])
pca_df['Symbol'] = df_clean['Symbol'].values
pca_df['GICS Sector'] = df_clean['GICS Sector'].values
pca_df['GICS Sub-Industry'] = df_clean['GICS Sub-Industry'].values

pca_df.to_csv("/content/outputs/pca_10_components.csv", index=False)

explained = pca.explained_variance_ratio_
print("✅ PCA complete!")
print(f"Total explained variance by 10 PCs: {explained.sum():.3f}")
print("Individual explained variance per component:\n", explained)


✅ PCA complete!
Total explained variance by 10 PCs: 0.136
Individual explained variance per component:
 [0.01755806 0.01473345 0.01437511 0.01401967 0.01334687 0.01314619
 0.0129055  0.01283214 0.01258975 0.01081644]


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd

wcss = []
silhouette_scores = []
K = range(2, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_result)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(pca_result, kmeans.labels_))

plt.figure(figsize=(6, 4))
plt.plot(K, wcss, 'o-', color='b')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.title('Elbow Method')
plt.grid(True)
plt.savefig("/content/outputs/elbow_wcss.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(6, 4))
plt.plot(K, silhouette_scores, 'o-', color='g')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Method')
plt.grid(True)
plt.savefig("/content/outputs/silhouette_scores.png", dpi=300, bbox_inches='tight')
plt.close()

best_k = silhouette_scores.index(max(silhouette_scores)) + 2
final_kmeans = KMeans(n_clusters=best_k, random_state=42)
labels = final_kmeans.fit_predict(pca_result)

pca_df["Cluster"] = labels
pca_df.to_csv("/content/outputs/pca10_clusters.csv", index=False)

plt.figure(figsize=(6, 5))
plt.scatter(pca_df["PC1"], pca_df["PC2"], c=labels, cmap='tab10', s=40)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"K-Means Clusters (k={best_k})")
plt.grid(True)
plt.savefig("/content/outputs/pc1_pc2_scatter_clusters.png", dpi=300, bbox_inches='tight')
plt.close()

print("✅ Clustering complete")
print(f"Best number of clusters: {best_k}")


✅ Clustering complete
Best number of clusters: 10


In [ ]:
pca_df['Cluster'].value_counts()


,count
Cluster,
1,96
3,72
7,72
4,59
2,46
5,32
6,31
0,30
8,19


In [ ]:
import sqlite3
conn = sqlite3.connect("/content/data/constituents.db")
print(pd.read_sql("SELECT COUNT(*) FROM constituents;", conn))


   COUNT(*)
0       503


In [18]:
!zip -r data.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/constituents.db (deflated 67%)


In [19]:
!zip -r outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/sql_top10_subindustries.csv (deflated 26%)
  adding: content/outputs/elbow_wcss.png (deflated 16%)
  adding: content/outputs/sql_sector_counts.csv (deflated 26%)
  adding: content/outputs/pc1_pc2_scatter_clusters.png (deflated 6%)
  adding: content/outputs/pca_10_components.csv (deflated 55%)
  adding: content/outputs/silhouette_scores.png (deflated 14%)
  adding: content/outputs/pca10_clusters.csv (deflated 55%)
  adding: content/outputs/sql_total_companies.csv (stored 0%)


In [20]:
!zip -r scripts.zip /content/scripts

  adding: content/scripts/ (stored 0%)
  adding: content/scripts/.ipynb_checkpoints/ (stored 0%)
  adding: content/scripts/main_sql_queries.sql (deflated 47%)
